### Importamos las dependencias

In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os


### Variables de entorno

In [12]:
load_dotenv()

api_key = os.getenv("API_KEY") 

### Subir un Documento

In [13]:

def upload_pdf(url: str):        
    try:
        loader = PyPDFLoader(url)
        loader = loader.lazy_load()

        text = ""

        for page in loader: 
            text += page.page_content + "\n"

        return text
    except Exception as e:
        print(e)
        return []
    


### Text Splitter para separar todo el contenido de mi documento

In [ ]:

def text_splitter(text): 

    text_splitter = CharacterTextSplitter(
    chunk_size = 7000,
    chunk_overlap = 200,
    separator="\n"
)
    texts = text_splitter.create_documents([text])
    print(texts)
    return texts



### Defino el modelo que utilizaré

In [15]:
embedding = OllamaEmbeddings(
    model = "nomic-embed-text"
)

### Creo mi base de datos vectorial donde se guardará mi embedding

In [16]:
def get_vector_store(name_collection: str): 
    
    vector_store = Chroma(
    collection_name= name_collection,
    embedding_function=embedding,
    persist_directory="./vectorstore"
)    
    return vector_store

### Creo el retrieval que devolverá la información en una busqueda de similitudes

In [17]:
def retrieval(input_user: str): 
    vector_store = get_vector_store("langchain")
    docs = vector_store.similarity_search(input_user)
    return docs

### Creamos el propt system para el modelo que vamos a utilizar

In [ ]:
prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder preguntas sobre Arquitectura de software y solo debes contestar si el contexto no está vacio.
    En caso de que no cuentes con la información solicitada responde "disculpe, pero la pregunta excede mi conocimiento" y si te preguntan algo fuera del contexto principal responde "No estoy programado para eso".
    Utiliza siempre el contexto proporcionado para responder.
    contexto = {contexto}
    pregunta del usuario: {input_user}
""")

### Creamos la función de respuesta que nos comunicará con nuestro agende de IA

In [ ]:
def response(input_user: str, contexto: str):
    llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.5-flash",
    temperature= 0.7
)

    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content

### Utilizamos las funciones para cargar el doc, aplicarle el text_splitter y guardar esos datos como embedding en la base de datos vectorial

In [20]:
loader = upload_pdf("mi_tp.pdf")
texts = text_splitter(loader)
vector_store = get_vector_store("langchain")

vector_store.add_documents(texts)

[Document(metadata={}, page_content='1. Investiga y explica con tus palabras cada uno de los siguientes conceptos(podés usar libros, videos, artículos, papers o apuntes de clase):○ ¿Qué es una red neuronal artificial?Una red neuronal artificial es una red neuronal que es artificial porque no es real, es creada por nosotros, es decir es creada por el humano pero no de nacimiento sino que fue creado por nosotros (Son nodos que se encargan de realizar una predicción a partir de datos que se le brindan a través de tensores que pueden estar más o menos abstraidos dependiendo de la libreria que se utiliza)○ ¿Qué función cumplen las capas (entrada, ocultas, salida)?Las funciones que cumoplen las capas son:Entrada: Reciben los datos que le mandamos (ej: pixeles de una imagen, números, etc).Oculta: Se encargan de hacer los cálculos y las transformaciones inmediatas.Salida: Devuelve la predicción o un mensaje final de los datos recibidos, como por ej: “gato”, “spam”, “positivo”, etc.○ ¿Qué es un

['33f8106e-dbf7-4d19-abda-0fd91dfb6e3c']

### Ponemos a prueba nuestro RAG

In [21]:

for l in range(2):
    input_user = input("Human: ")
    print(input_user)

    docs = retrieval(input_user=input_user)
    print(docs)

    for chunk in response(input_user=input_user, contexto=docs):
        print(chunk, end=" ", flush=True)
    

que es un neurona
[Document(id='33f8106e-dbf7-4d19-abda-0fd91dfb6e3c', metadata={}, page_content='1. Investiga y explica con tus palabras cada uno de los siguientes conceptos(podés usar libros, videos, artículos, papers o apuntes de clase):○ ¿Qué es una red neuronal artificial?Una red neuronal artificial es una red neuronal que es artificial porque no es real, es creada por nosotros, es decir es creada por el humano pero no de nacimiento sino que fue creado por nosotros (Son nodos que se encargan de realizar una predicción a partir de datos que se le brindan a través de tensores que pueden estar más o menos abstraidos dependiendo de la libreria que se utiliza)○ ¿Qué función cumplen las capas (entrada, ocultas, salida)?Las funciones que cumoplen las capas son:Entrada: Reciben los datos que le mandamos (ej: pixeles de una imagen, números, etc).Oculta: Se encargan de hacer los cálculos y las transformaciones inmediatas.Salida: Devuelve la predicción o un mensaje final de los datos recibid